In [4]:
from importlib import reload
import pandas as pd

import psycopg2
import settings as stt

from itables import init_notebook_mode, show
import re

from time import strftime, gmtime

In [5]:
import postgresql_functions as pgf

In [6]:
# reload(pgf)

## Sentences : A studied B at C from t1 to t2

Michael Mästlin studied mathematics and astronomy at Tübingen University - 1568 to 1571.

_

In [7]:
### connect to the local database
conn = psycopg2.connect(host="localhost", port = 5432, database="espace_intellectuel", 
                        user="postgres", password=stt.dbw)
#conn

In [8]:
q1 = """
with tw1 as (
select pk_sentence,st_id, fk_mathshistory, sentence 
from mathshistory.coref_sentence cs
where sentence ~ 'University'
--limit 200
), tw2 as (
select ct.t_text tx, concat_ws(' ', ct.t_index, ct.t_pos_, ct.t_dep_, ct.t_head_text, ct.t_head_i),
ct1.t_text tx1 , concat_ws(' ', ct1.t_index, ct1.t_pos_, ct1.t_dep_, ct1.t_head_text, ct1.t_head_i) ,
ct2.t_text tx2, concat_ws(' ', ct2.t_index, ct2.t_pos_, ct2.t_dep_, ct2.t_head_text, ct2.t_head_i) ,
ct3.t_text tx3, concat_ws(' ', ct3.t_index, ct3.t_pos_, ct3.t_dep_, ct3.t_head_text, ct3.t_head_i) , 
ct4.t_text tx4, concat_ws(' ', ct4.t_index, ct4.t_pos_, ct4.t_dep_, ct4.t_head_text, ct4.t_head_i) ,
ct5.t_text tx5, concat_ws(' ', ct5.t_index, ct5.t_pos_, ct5.t_dep_, ct5.t_head_text, ct5.t_head_i)
from mathshistory.coref_token ct 
join tw1 on ct.fk_sentence = tw1.pk_sentence
join mathshistory.coref_token ct1 on ct1.fk_sentence = tw1.pk_sentence and ct1.t_head_i = ct.t_index 
join mathshistory.coref_token ct2 on ct2.fk_sentence = tw1.pk_sentence and ct2.t_index = ct1.t_index +1
left join mathshistory.coref_token ct3 on ct3.fk_sentence = tw1.pk_sentence 
		and ct3.t_index = ct2.t_index +1 and ct3.t_pos_ = 'PROPN'
left join mathshistory.coref_token ct4 on ct4.fk_sentence = tw1.pk_sentence 
		and ct4.t_index = ct3.t_index +1 and ct4.t_pos_ = 'PROPN'
left join mathshistory.coref_token ct5 on ct5.fk_sentence = tw1.pk_sentence 
		and ct5.t_index = ct4.t_index +1 and ct5.t_pos_ = 'PROPN'
where ct.t_text = 'University'
and ct1.t_dep_ ='prep'
and ct1.t_text in ('of')) --, 'at', 'in'
select concat_ws(' ', tx, tx1, tx2,tx3,tx4, tx5) ne, count(*) eff
from tw2
group by tx, tx1, tx2, tx3,tx3,tx4, tx5
order by eff desc;
"""

In [9]:
result = pgf.sql_explore(q1, conn)
# print(f'Lines count: {len(result[0])}, errors count: {len(result[1])}, \nFirst lines: {result[0][:5]}')

In [10]:
ne = pd.DataFrame(result[0])
ne.columns = ['name', 'freq']
ne.head()

,name,freq
0,University of California,357
1,University of Berlin,337
2,University of Chicago,315
3,University of London,259
4,University of Cambridge,253


In [11]:
### https://github.com/mwouts/itables/blob/main/docs/advanced_parameters.md
show(ne.sort_values(by='freq', ascending=False), classes="display",

     columnDefs=[{"className": "dt-left", "targets": "_all"}],
     scrollY="400px", scrollCollapse=True, paging=False, column_filters="footer", dom="lrtip")

In [12]:
q = """
with tw1 as (
select pk_sentence,st_id, fk_mathshistory, sentence 
from mathshistory.coref_sentence cs
where sentence ~ 'University'
--limit 200
), tw2 as (
select h1.t_head_text h1_text, concat_ws(' ', h1.t_head_pos_,h1.t_dep_) h1_pos, 
			ct.t_head_text h_text, ct.t_head_pos_ h_pos, ct.t_text tx, concat_ws(' ', ct.t_index, ct.t_pos_, ct.t_dep_, ct.t_head_text, ct.t_head_i, 
			ct.t_head_text, ct.t_head_pos_),
ct1.t_text tx1 , concat_ws(' ', ct1.t_index, ct1.t_pos_, ct1.t_dep_, ct1.t_head_text, ct1.t_head_i) ,
ct2.t_text tx2, concat_ws(' ', ct2.t_index, ct2.t_pos_, ct2.t_dep_, ct2.t_head_text, ct2.t_head_i) ,
ct3.t_text tx3, concat_ws(' ', ct3.t_index, ct3.t_pos_, ct3.t_dep_, ct3.t_head_text, ct3.t_head_i) , 
ct4.t_text tx4, concat_ws(' ', ct4.t_index, ct4.t_pos_, ct4.t_dep_, ct4.t_head_text, ct4.t_head_i) ,
ct5.t_text tx5, concat_ws(' ', ct5.t_index, ct5.t_pos_, ct5.t_dep_, ct5.t_head_text, ct5.t_head_i)
from mathshistory.coref_token ct 
join tw1 on ct.fk_sentence = tw1.pk_sentence
join mathshistory.coref_token ct1 on ct1.fk_sentence = tw1.pk_sentence and ct1.t_head_i = ct.t_index 
--head 2nd level 
join mathshistory.coref_token h1 on h1.fk_sentence = tw1.pk_sentence and h1.t_index = ct.t_head_i 
join mathshistory.coref_token ct2 on ct2.fk_sentence = tw1.pk_sentence and ct2.t_index = ct1.t_index +1
left join mathshistory.coref_token ct3 on ct3.fk_sentence = tw1.pk_sentence 
		and ct3.t_index = ct2.t_index +1 and ct3.t_pos_ = 'PROPN'
left join mathshistory.coref_token ct4 on ct4.fk_sentence = tw1.pk_sentence 
		and ct4.t_index = ct3.t_index +1 and ct4.t_pos_ = 'PROPN'
left join mathshistory.coref_token ct5 on ct5.fk_sentence = tw1.pk_sentence 
		and ct5.t_index = ct4.t_index +1 and ct5.t_pos_ = 'PROPN'
where ct.t_text = 'University'
and ct1.t_dep_ ='prep'
and ct1.t_text in ('of')) --, 'at', 'in'
select h1_text, h1_pos, h_text, h_pos, tx, tx1, tx2,tx3,tx4, tx5, count(*) eff
from tw2
group by h1_text, h1_pos, h_text, h_pos, tx, tx1, tx2, tx3,tx3,tx4, tx5
order by eff desc;
"""

In [13]:
result = pgf.sql_explore(q, conn)
# print(f'Lines count: {len(result[0])}, errors count: {len(result[1])}, \nFirst lines: {result[0][:5]}')

In [16]:
univ = pd.DataFrame(result[0])
univ.columns=['h1_text', 'h1_pos', 'h_text', 'h_pos', 'tx', 'tx1', 'tx2','tx3','tx4', 'tx5', 'eff']
univ.head()

,h1_text,h1_pos,h_text,h_pos,tx,tx1,tx2,tx3,tx4,tx5,eff
0,entered,VERB ROOT,entered,VERB,University,of,Berlin,None,None,None,25
1,studied,VERB prep,at,ADP,University,of,Berlin,None,None,None,22
2,professor,NOUN prep,at,ADP,University,of,California,None,None,None,21
3,matriculated,VERB prep,at,ADP,University,of,Edinburgh,None,None,None,19
4,went,VERB prep,to,ADP,University,of,Göttingen,None,None,None,17


In [17]:
### https://github.com/mwouts/itables/blob/main/docs/advanced_parameters.md
show(univ.sort_values(by='eff', ascending=False), classes="display",

     columnDefs=[{"className": "dt-left", "targets": "_all"}],
     scrollY="400px", scrollCollapse=True, paging=False, column_filters="footer", dom="lrtip")